In [40]:
with open("day10.txt", "r") as f:
    data = f.read()

In [41]:
print(data)

..77LJJ7|7.|F-L77.F77FLF-L-777L7.77.F7-77-F|-7-F-FL-LF7F77FFF-7F7F7F-7F7FFF777|.FL|.F|.FFF-7|FL777FF.FFF77F--L-77--|FJFFF7FLF-|-JF7J-F--7-77
-||L7L-7LJ7L7J|LJ.|.J7|L--7||7FFFJ..-F7.7F7JF--J7|J|||LJ|F-7L7LJLJ||FJF-7FJL77J-L-77F|F-J.LF77-||F-F--F|--777|7|||7JL7LJF7-JJFLFJF|J.|||L-7J
L|7L7F--J.|-JF-7J-77F7J-JFFJ-|-J|L-L7J|.L||L|J|.|L-77L-7||FJFJF7F-J||-L7LJF-JJ.LJ--7JL77LJ7LLJ-LJJL7-7J|J.|F-J--J.7|FJ.|||LJF7.J---7-J-|.|F7
.J7.JJJ-F|JF-7.|7-L|7JL7J7.FFLJF77-LL.L-FF7-F7-F7.FF7JFJ|||FJFJLJF-J|F7L7FJJ|..FJFL-.L|-J.|-7||JJJL|FL-JJF|.L-777LF-F7--.J..FJ|JL|-|7FFJ-F||
FF--|J.|.LF|F7-L---LFF-J-J-J-7LL|LFJ|7J-LL-.LJFF|-FJL7L7||LJFJF-7L-7LJL-JL-7J.F-FJL|J.LL7J.-.|.L|7.F|.|.FLJ-L.|L7J..LLLJ7|FFLF-.L-.||7L-F|L7
L77F|.L---JLJL|77LFJ|.F--LJF|J|F|-|LL7L7FLL7-FF--7L-7L7||L7FJLL7|JL|F----7FJJ-LFFJF7L7JLF77..7-J|LF-JFFLF.||JLJ.|.FJ.LL.L|F7JLF-77-|L7J7||F|
LLFJJ.JJ.J--7-L7|-|J|L7L-J-LJ7F-7||7|.-L-.LFF7L7FJF7|FJ|L-JL-7FJL7FJL---7|L--7.L--7J7|.F7J77..|7F-L7F-L-7FJ77.F7L-F7F7J.||LL7L7.FJFFF7---FLJ
7J.|-7F7.F7L|

In [42]:
# The pipes are arranged in a two-dimensional grid of tiles:

# | is a vertical pipe connecting north and south.
# - is a horizontal pipe connecting east and west.
# L is a 90-degree bend connecting north and east.
# J is a 90-degree bend connecting north and west.
# 7 is a 90-degree bend connecting south and west.
# F is a 90-degree bend connecting south and east.
# . is ground; there is no pipe in this tile.
# S is the starting position of the animal; there is a pipe on this tile, but your sketch doesn't show what shape the pipe has.
# Based on the acoustics of the animal's scurrying, you're confident the pipe that contains the animal is one large, continuous loop.

# 1. convert into a graph with potential edges
# 2. filter the edges that are connected to both ends so they are true edges
# 3. find the length of the loop that has S

lines = data.split("\n")
height = len(lines)
width = len(lines[0])

# each node: (up, down, left, right)
# graph: x,y -> node
graph = []
for y in range(height):
    graph.append([])
    for x in range(width):
        graph[y].append([False, False, False, False])

start_x, start_y = 0,0

# stage 1: convert into a graph with potential edges
for row_idx, row in enumerate(lines):
    for col_idx, value in enumerate(row):

        if value == "S":
            start_x, start_y = col_idx, row_idx

        if value == "|":
            graph[row_idx][col_idx][0] = True
            graph[row_idx][col_idx][1] = True
        elif value == "-":
            graph[row_idx][col_idx][2] = True
            graph[row_idx][col_idx][3] = True
        elif value == "L":
            graph[row_idx][col_idx][0] = True
            graph[row_idx][col_idx][3] = True
        elif value == "J":
            graph[row_idx][col_idx][0] = True
            graph[row_idx][col_idx][2] = True
        elif value == "7":
            graph[row_idx][col_idx][1] = True
            graph[row_idx][col_idx][2] = True
        elif value == "F":
            graph[row_idx][col_idx][1] = True
            graph[row_idx][col_idx][3] = True

# stage 1.1: fix S node according to neighbouring edges
# just for now
graph[start_y][start_x][0] = True
graph[start_y][start_x][1] = True
graph[start_y][start_x][2] = True
graph[start_y][start_x][3] = True

# stage 2: filter the edges that are connected to both ends so they are true edges
for y in range(height):
    for x in range(width):
        try:
            if graph[y][x][0]:
                if not( y > 0 and graph[y-1][x][1]):
                    graph[y][x][0] = False
                    graph[y-1][x][1] = False
            if graph[y][x][1]:
                if not (y < height-1 and graph[y+1][x][0]):
                    graph[y][x][1] = False
                    graph[y+1][x][0] = False
            if graph[y][x][2]:
                if not(x > 0 and graph[y][x-1][3]):
                    graph[y][x][2] = False
                    graph[y][x-1][3] = False
            if graph[y][x][3]:
                if not(x < width-1 and graph[y][x+1][2]):
                    graph[y][x][3] = False
                    graph[y][x+1][2] = False
        except IndexError:
            pass
            # print("IndexError", x, y)


# stage 3: find the length of the loop that has S
cur_x, cur_y = start_x, start_y
step = 0
while True:
    step += 1

    # try to decide which way to go
    to_disable_egde = None
    if graph[cur_y][cur_x][0]:
        # up
        cur_y -= 1
        to_disable_egde = 1
        print("up")
    elif graph[cur_y][cur_x][1]:
        # down
        cur_y += 1
        to_disable_egde = 0
        print("down")
    elif graph[cur_y][cur_x][2]:
        # left
        cur_x -= 1
        to_disable_egde = 3
        print("left")
    elif graph[cur_y][cur_x][3]:
        # right
        cur_x += 1
        to_disable_egde = 2
        print("right")
    else:
        print("no way to go")
        break
    
    # when arrived at next node, need to disable the source edge to prevent infinite loop
    graph[cur_y][cur_x][to_disable_egde] = False


    if (cur_x, cur_y) == (start_x, start_y):
        print("loop found")
        break

print("loop len", step)

print("res", step/2)

up
right
up
right
down
down
right
up
up
up
left
up
right
up
up
left
up
left
left
left
up
right
right
up
left
left
left
left
up
up
right
down
right
up
up
left
up
right
up
right
down
down
down
down
right
up
up
up
up
up
up
up
up
right
down
down
down
down
down
down
down
down
right
up
up
up
up
up
up
up
up
right
down
down
down
down
down
right
up
up
up
up
up
up
right
down
down
right
down
left
down
right
down
left
down
down
left
left
down
down
down
left
left
down
right
down
right
down
down
right
up
up
up
up
up
up
right
down
down
down
right
up
up
right
down
down
down
left
left
down
down
right
up
right
down
down
left
left
left
left
down
right
down
left
down
down
right
down
left
left
down
right
down
right
down
left
down
right
down
left
left
down
down
right
up
right
down
down
down
down
left
down
right
down
left
down
right
down
down
left
left
down
right
right
down
left
left
down
right
right
down
right
up
up
up
up
up
up
right
down
down
down
right
up
up
up
up
left
left
up
up
up
up
up
up
up
up
up
up
u

In [106]:
with open("day10.txt", "r") as f:
    data = f.read()

In [107]:
print(data)

..77LJJ7|7.|F-L77.F77FLF-L-777L7.77.F7-77-F|-7-F-FL-LF7F77FFF-7F7F7F-7F7FFF777|.FL|.F|.FFF-7|FL777FF.FFF77F--L-77--|FJFFF7FLF-|-JF7J-F--7-77
-||L7L-7LJ7L7J|LJ.|.J7|L--7||7FFFJ..-F7.7F7JF--J7|J|||LJ|F-7L7LJLJ||FJF-7FJL77J-L-77F|F-J.LF77-||F-F--F|--777|7|||7JL7LJF7-JJFLFJF|J.|||L-7J
L|7L7F--J.|-JF-7J-77F7J-JFFJ-|-J|L-L7J|.L||L|J|.|L-77L-7||FJFJF7F-J||-L7LJF-JJ.LJ--7JL77LJ7LLJ-LJJL7-7J|J.|F-J--J.7|FJ.|||LJF7.J---7-J-|.|F7
.J7.JJJ-F|JF-7.|7-L|7JL7J7.FFLJF77-LL.L-FF7-F7-F7.FF7JFJ|||FJFJLJF-J|F7L7FJJ|..FJFL-.L|-J.|-7||JJJL|FL-JJF|.L-777LF-F7--.J..FJ|JL|-|7FFJ-F||
FF--|J.|.LF|F7-L---LFF-J-J-J-7LL|LFJ|7J-LL-.LJFF|-FJL7L7||LJFJF-7L-7LJL-JL-7J.F-FJL|J.LL7J.-.|.L|7.F|.|.FLJ-L.|L7J..LLLJ7|FFLF-.L-.||7L-F|L7
L77F|.L---JLJL|77LFJ|.F--LJF|J|F|-|LL7L7FLL7-FF--7L-7L7||L7FJLL7|JL|F----7FJJ-LFFJF7L7JLF77..7-J|LF-JFFLF.||JLJ.|.FJ.LL.L|F7JLF-77-|L7J7||F|
LLFJJ.JJ.J--7-L7|-|J|L7L-J-LJ7F-7||7|.-L-.LFF7L7FJF7|FJ|L-JL-7FJL7FJL---7|L--7.L--7J7|.F7J77..|7F-L7F-L-7FJ77.F7L-F7F7J.||LL7L7.FJFFF7---FLJ
7J.|-7F7.F7L|

In [108]:
# The pipes are arranged in a two-dimensional grid of tiles:

# | is a vertical pipe connecting north and south.
# - is a horizontal pipe connecting east and west.
# L is a 90-degree bend connecting north and east.
# J is a 90-degree bend connecting north and west.
# 7 is a 90-degree bend connecting south and west.
# F is a 90-degree bend connecting south and east.
# . is ground; there is no pipe in this tile.
# S is the starting position of the animal; there is a pipe on this tile, but your sketch doesn't show what shape the pipe has.
# Based on the acoustics of the animal's scurrying, you're confident the pipe that contains the animal is one large, continuous loop.

# 1. convert into a graph with potential edges
# 2. filter the edges that are connected to both ends so they are true edges
# 3. find the length of the loop that has S

lines = data.split("\n")
height = len(lines)
width = len(lines[0])

# each node: (up, down, left, right)
# graph: x,y -> node
graph = []
for y in range(height):
    graph.append([])
    for x in range(width):
        graph[y].append([False, False, False, False])

start_x, start_y = 0,0

# stage 1: convert into a graph with potential edges
for row_idx, row in enumerate(lines):
    for col_idx, value in enumerate(row):

        if value == "S":
            start_x, start_y = col_idx, row_idx

        if value == "|":
            graph[row_idx][col_idx][0] = True
            graph[row_idx][col_idx][1] = True
        elif value == "-":
            graph[row_idx][col_idx][2] = True
            graph[row_idx][col_idx][3] = True
        elif value == "L":
            graph[row_idx][col_idx][0] = True
            graph[row_idx][col_idx][3] = True
        elif value == "J":
            graph[row_idx][col_idx][0] = True
            graph[row_idx][col_idx][2] = True
        elif value == "7":
            graph[row_idx][col_idx][1] = True
            graph[row_idx][col_idx][2] = True
        elif value == "F":
            graph[row_idx][col_idx][1] = True
            graph[row_idx][col_idx][3] = True

# stage 1.1: fix S node according to neighbouring edges
# just for now
graph[start_y][start_x][0] = True
graph[start_y][start_x][1] = True
graph[start_y][start_x][2] = True
graph[start_y][start_x][3] = True

# stage 2: filter the edges that are connected to both ends so they are true edges
for y in range(height):
    for x in range(width):
        try:
            if graph[y][x][0]:
                if not( y > 0 and graph[y-1][x][1]):
                    graph[y][x][0] = False
                    graph[y-1][x][1] = False
            if graph[y][x][1]:
                if not (y < height-1 and graph[y+1][x][0]):
                    graph[y][x][1] = False
                    graph[y+1][x][0] = False
            if graph[y][x][2]:
                if not(x > 0 and graph[y][x-1][3]):
                    graph[y][x][2] = False
                    graph[y][x-1][3] = False
            if graph[y][x][3]:
                if not(x < width-1 and graph[y][x+1][2]):
                    graph[y][x][3] = False
                    graph[y][x+1][2] = False
        except IndexError:
            pass
            # print("IndexError", x, y)


# stage 3: find the length of the loop that has S
cur_x, cur_y = start_x, start_y
step = 0

boundary_points = []

while True:
    step += 1
    boundary_points.append((cur_y, cur_x))

    # try to decide which way to go
    to_disable_egde = None
    if graph[cur_y][cur_x][0]:
        # up
        cur_y -= 1
        to_disable_egde = 1
        print("up")
    elif graph[cur_y][cur_x][1]:
        # down
        cur_y += 1
        to_disable_egde = 0
        print("down")
    elif graph[cur_y][cur_x][2]:
        # left
        cur_x -= 1
        to_disable_egde = 3
        print("left")
    elif graph[cur_y][cur_x][3]:
        # right
        cur_x += 1
        to_disable_egde = 2
        print("right")
    else:
        print("no way to go")
        break
    
    # when arrived at next node, need to disable the source edge to prevent infinite loop
    graph[cur_y][cur_x][to_disable_egde] = False


    if (cur_x, cur_y) == (start_x, start_y):
        print("loop found")
        break

print("loop len", step)

print("res", step/2)

up
right
up
right
down
down
right
up
up
up
left
up
right
up
up
left
up
left
left
left
up
right
right
up
left
left
left
left
up
up
right
down
right
up
up
left
up
right
up
right
down
down
down
down
right
up
up
up
up
up
up
up
up
right
down
down
down
down
down
down
down
down
right
up
up
up
up
up
up
up
up
right
down
down
down
down
down
right
up
up
up
up
up
up
right
down
down
right
down
left
down
right
down
left
down
down
left
left
down
down
down
left
left
down
right
down
right
down
down
right
up
up
up
up
up
up
right
down
down
down
right
up
up
right
down
down
down
left
left
down
down
right
up
right
down
down
left
left
left
left
down
right
down
left
down
down
right
down
left
left
down
right
down
right
down
left
down
right
down
left
left
down
down
right
up
right
down
down
down
down
left
down
right
down
left
down
right
down
down
left
left
down
right
right
down
left
left
down
right
right
down
right
up
up
up
up
up
up
right
down
down
down
right
up
up
up
up
left
left
up
up
up
up
up
up
up
up
up
up
u

In [109]:
# rebuild graph
# 图被改了，需要重建

for row_idx, row in enumerate(lines):
    for col_idx, value in enumerate(row):

        if value == "S":
            start_x, start_y = col_idx, row_idx

        if value == "|":
            graph[row_idx][col_idx][0] = True
            graph[row_idx][col_idx][1] = True
        elif value == "-":
            graph[row_idx][col_idx][2] = True
            graph[row_idx][col_idx][3] = True
        elif value == "L":
            graph[row_idx][col_idx][0] = True
            graph[row_idx][col_idx][3] = True
        elif value == "J":
            graph[row_idx][col_idx][0] = True
            graph[row_idx][col_idx][2] = True
        elif value == "7":
            graph[row_idx][col_idx][1] = True
            graph[row_idx][col_idx][2] = True
        elif value == "F":
            graph[row_idx][col_idx][1] = True
            graph[row_idx][col_idx][3] = True

# stage 1.1: fix S node according to neighbouring edges
# just for now
graph[start_y][start_x][0] = True
graph[start_y][start_x][1] = True
graph[start_y][start_x][2] = True
graph[start_y][start_x][3] = True

# stage 2: filter the edges that are connected to both ends so they are true edges
for y in range(height):
    for x in range(width):
        try:
            if graph[y][x][0]:
                if not( y > 0 and graph[y-1][x][1]):
                    graph[y][x][0] = False
                    graph[y-1][x][1] = False
            if graph[y][x][1]:
                if not (y < height-1 and graph[y+1][x][0]):
                    graph[y][x][1] = False
                    graph[y+1][x][0] = False
            if graph[y][x][2]:
                if not(x > 0 and graph[y][x-1][3]):
                    graph[y][x][2] = False
                    graph[y][x-1][3] = False
            if graph[y][x][3]:
                if not(x < width-1 and graph[y][x+1][2]):
                    graph[y][x][3] = False
                    graph[y][x+1][2] = False
        except IndexError:
            pass
            # print("IndexError", x, y)

In [110]:
# use ray-casting (to the right) to check if a point is in boundary or not
inside_count = 0

# for y in [1]:
for y in range(height):
    for x in range(width):
        
        # for each point

        # already on boundary, don't count
        if (y, x) in boundary_points:
            continue

        # shoot to the right
        intersection_count = 0
        for shoot_x in range(x+1, width):
            # print("shooting", y, shoot_x)
            # 视作一个向右上角的小角度射线，只会经过上边
            if (y, shoot_x) in boundary_points and graph[y][shoot_x][0]:
                # print("intersection", y, shoot_x)
                intersection_count += 1
        
        if intersection_count % 2 == 1:
            print("inside", y, x, intersection_count)
            inside_count += 1

inside_count

inside 36 43 57
inside 36 53 49
inside 36 81 27
inside 36 82 27
inside 36 102 9
inside 38 37 61
inside 38 42 57
inside 38 50 51
inside 38 63 43
inside 38 68 39
inside 39 39 47
inside 39 96 9
inside 39 100 7
inside 40 41 53
inside 41 63 37
inside 41 73 31
inside 41 97 13
inside 42 36 53
inside 43 66 31
inside 44 56 31
inside 44 102 9
inside 45 40 49
inside 45 76 27
inside 46 57 41
inside 47 92 19
inside 48 74 27
inside 48 99 15
inside 49 91 19
inside 50 87 13
inside 52 45 41
inside 52 50 41
inside 52 69 29
inside 53 61 43
inside 54 38 41
inside 54 51 39
inside 54 54 39
inside 54 64 33
inside 54 80 25
inside 54 90 19
inside 55 43 33
inside 55 48 31
inside 55 99 9
inside 56 59 25
inside 57 64 21
inside 57 102 7
inside 58 63 25
inside 58 64 25
inside 58 65 25
inside 58 66 25
inside 58 75 19
inside 58 78 17
inside 59 37 41
inside 59 40 39
inside 59 44 37
inside 59 49 37
inside 59 61 33
inside 59 62 33
inside 59 63 33
inside 59 64 33
inside 59 65 33
inside 59 88 21
inside 60 60 37
inside 60 

471